In [1]:
import pandas as pd
import numpy as np

btc_df = pd.read_csv('btc_os_cnn_data.csv', parse_dates=['open_time'])
btc_df = btc_df.sort_values('open_time').reset_index(drop=True)
btc_df = btc_df[['open', 'high', 'low', 'close', 'volume']]
btc_df['label'] = (btc_df['open'] < btc_df['close']).astype(int)
btc_df.head()

,open,high,low,close,volume,label
0,36060.5,36060.5,35807.9,35815.6,10.423,0
1,35815.5,35939.5,35717.3,35722.0,9.339,0
2,35715.6,35849.5,35715.6,35723.4,3.884,1
3,35735.1,35735.1,35720.0,35727.6,1.684,0
4,35727.2,35850.0,35714.0,35831.6,3.474,1


In [2]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length):
    sequences = []
    data_size = len(input_data)
    print(data_size)
    for i in range(data_size - sequence_length):
        seq = input_data[i:i+sequence_length]
        label = input_data.iloc[i + sequence_length][target_column]
        
        ##########################################################################
        # I changed this line
        ##########################################################################
        numpy_train = np.expand_dims(seq.to_numpy().T[0:-1],0)
        ##########################################################################

        sequences.append((numpy_train, label))
    return sequences

sequences = create_sequences(btc_df,'label',200)

127788


In [3]:
print(sequences[0][0].shape)

(1, 5, 200)


In [4]:
data = np.concatenate([i[0] for i in sequences], axis=0)
print(data.shape)
label = np.array([i[1] for i in sequences])
print(label.shape)

(127588, 5, 200)
(127588,)


In [5]:
# I like to save them as numpy then, we don't need to pre-process for each time we run.

np.save('data',data)
np.save('label',label)

In [6]:
# load data
data = np.float32(np.load('./data.npy'))
label = np.int64(np.load('./label.npy'))


# 80% as train data, and 20% as test data
train_ration = 0.8
cut = int(train_ration*data.shape[0])


X_train = data[0:cut]
y_train = label[0:cut]
X_test = data[cut:-1]
y_test = label[cut:-1]
    
print('train data shape', X_train.shape)
print('train label shape',y_train.shape)
print('test data shape',X_test.shape)
print('test label shape',y_test.shape)

print('unique train label',np.unique(y_train))
print('unique test label',np.unique(y_test))

train data shape (102070, 5, 200)
train label shape (102070,)
test data shape (25517, 5, 200)
test label shape (25517,)
unique train label [0 1]
unique test label [0 1]
